# --- Day 7: No Space Left On Device ---



In [4]:
# Load data
filename = "inputs/day7-example.txt"
with open(filename) as infile:
    terminal_output = [line.strip() for line in infile]
    

In [123]:
class Tree(object):
    def __init__(self, name='/', children=None, files=None):
        self.name = name
        self.children = []
        self.files = []
        self.size_files = 0
        self.size_subdirs = 0
        self.parent=None
        
        if children is not None:
            for child in children:
                self.add_child(child)
                
        if files is not None:
            for file in files:
                self.add_file(file)     
                
    def __repr__(self):
        return self.name
    
    def add_child(self, node):
        assert isinstance(node, Tree)
        node.parent=self
        print(f"Adding child {node.name} with parent {node.parent}")
        self.children.append(node)
        
    def add_file(self, file):
        self.files.append(file)   
        file_size = file.get(list(file.keys())[0])
        print(f"Adding file {file} with size {file_size} to directory {self.name}")
        self.size_files += file.get(list(file.keys())[0])


def traverse_filesystem(terminal_data):
    # Sketch of how to solve it...
    # 
    # Parse terminal data into a filesystem tree
    #
    # A node is defined by the sublist of terminal data 
    # starting with a pair of entries
    #     ["$ cd DIRECTORY", "$ ls"] 
    # ending with the corresponding "$ cd .." or end of 
    # the list (if there are no remaining commands)
    #
    # This means we know the number of nodes - it is the 
    # number of occurrences of "$ ls" in the terminal data
    
        
    file_tree = Tree()
    

    def process_node(file_tree, terminal_data, small_subdirs_total):
        print(f"\ndirectory\t:{file_tree.name}")
        #dir_size = file_tree.size_files
        dir_size = 0
        if len(terminal_data) > 0:
            
            # @TODO: return parent if found "cd .."
            print(f"\n{terminal_data}\n")
            # Stuff that happens when we're processing a node ...
            # Find first occurrence of "$ cd DIRECTORY" followed by "$ ls"
            indices_first_cd = [i for i in range(0, len(terminal_data)-1) if terminal_data[i] == f"$ cd {file_tree.name}" and terminal_data[i+1] == "$ ls"]
            index_first_cd = min(indices_first_cd) if len(indices_first_cd) > 0 else 0
            
            #print(indices_first_cd)
            #print(index_next_cdup)
            indices_next_command = [i for i in range(index_first_cd+2, len(terminal_data)-1) if terminal_data[i].startswith("$ ")]           
            index_next_cdup = min([i for i in range(index_first_cd, len(terminal_data)-1) if terminal_data[i] == "$ cd .."]) if "$ cd .." in terminal_data else len(terminal_data)               
            index_next_command = min(indices_next_command) if len(indices_next_command) > 0 else len(terminal_data)
            # print(f"index_first_cd (1): {index_first_cd}")
            # print(f"index_next_cdup (1): {index_next_cdup}")            
            # print(f"index_next_command (1): {index_next_command}")
            if index_next_cdup <= index_first_cd and "$ cd .." in terminal_data:
                #print(terminal_data[index_next_cdup:])
                return process_node(file_tree.parent, terminal_data[index_next_cdup+1:], small_subdirs_total)
                #return file_tree.parent, terminal_data[index_next_cdup+1:], 0, small_subdirs_total
            
            #if terminal_data[index_next_command] == "$ cd ..":
            #    return file_tree.parent, terminal_data[index_next_command:], 0, small_subdirs_total
            
        
            node_listing = terminal_data[index_first_cd+2:index_next_command]
            #print(node_listing)
            node_files = [x for x in node_listing if not x.startswith("dir ")]
            node_dirs = [x for x in node_listing if x.startswith("dir ")]
            #print(f"len(node_dirs): {len(node_dirs)}")
            #print(f"index_next_command (2): {index_next_command}")            
            #print(f"files: {node_files}")
            #print(f"subdirs: {node_dirs}")
            
            #file_tree = Tree(name = terminal_data[index_first_cd].split("cd ")[1])
            for file in node_files:
                file_tree.add_file({file.split(" ")[1]: int(file.split(" ")[0])})
            
            if (len(node_dirs) == 0) and (index_next_cdup <= index_next_command) and "$ cd .." in terminal_data:
                #print(terminal_data[index_next_cdup:])
                #print(file_tree.name)                
                #print(file_tree.parent)
                #return file_tree.parent, terminal_data[index_next_cdup+1:], 0, small_subdirs_total                
                return process_node(file_tree.parent, terminal_data[index_next_cdup+1:], small_subdirs_total)
                #return file_tree.parent, terminal_data[index_next_cdup+1:], 0, small_subdirs_total
                
            
            # if len(node_dirs) == 0:
            #     print("this time")
            #     del terminal_data[index_first_cd:index_next_command]
            
            for subdir in node_dirs:
                subdir_tree = Tree(name = subdir.split("dir ")[1])
                file_tree.add_child(subdir_tree)                
                subdir_tree, terminal_data, subdir_size, small_subdirs_total = process_node(subdir_tree, terminal_data[index_next_command:], small_subdirs_total)
                dir_size += subdir_size
                #file_tree.add_child(subdir_tree)
            #print(f"dir_size: {dir_size}")
            
            if dir_size <= 100000:
                small_subdirs_total += dir_size
                

        
        return file_tree, terminal_data, dir_size, small_subdirs_total

    small_subdirs_total=0
    file_tree, _, _, small_subdirs_total = process_node(file_tree, terminal_data, small_subdirs_total)
    
    print(f"small_subdirs_total: {small_subdirs_total}")
            
                
    
    return file_tree
    
tree = traverse_filesystem(terminal_output.copy())
#print(tree.name)
#print(tree.children)
# print(tree.files)
# print(tree.size_files)



directory	:/

['$ cd /', '$ ls', 'dir a', '14848514 b.txt', '8504156 c.dat', 'dir d', '$ cd a', '$ ls', 'dir e', '29116 f', '2557 g', '62596 h.lst', '$ cd e', '$ ls', '584 i', '$ cd ..', '$ cd ..', '$ cd d', '$ ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']

Adding file {'b.txt': 14848514} with size 14848514 to directory /
Adding file {'c.dat': 8504156} with size 8504156 to directory /
Adding child a with parent /

directory	:a

['$ cd a', '$ ls', 'dir e', '29116 f', '2557 g', '62596 h.lst', '$ cd e', '$ ls', '584 i', '$ cd ..', '$ cd ..', '$ cd d', '$ ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']

Adding file {'f': 29116} with size 29116 to directory a
Adding file {'g': 2557} with size 2557 to directory a
Adding file {'h.lst': 62596} with size 62596 to directory a
Adding child e with parent a

directory	:e

['$ cd e', '$ ls', '584 i', '$ cd ..', '$ cd ..', '$ cd d', '$ ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']

Adding file {